<a href="https://colab.research.google.com/github/yasstake/rusty-bot/blob/main/tutorial/basilbot_backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# バジルさんの記事「Binanceで数か月運用していた高頻度botの紹介」を実装してみる。
https://note.com/kkngo/n/n13fb59bacc95?sub_rt=share_sb

## バックテスト　＆　フォーワードテスト

[テストネットによる本番運用はこちら](.//basilbot_real_run_testnet.ipynb)

## ロジック
バジルさんのロジックを引用すると以下のとおり

* 取引頻度
    約定履歴から作成した3秒足ごと
* エントリー/イグジット条件
    * 常に買いから入り、終値*0.9995など決め打ちでの買い指値（数字は仮）
    * イグジットも終値*1.0005など決め打ちでの売り指値（数字は仮）

* ピラミッディングやドテンの類はなし
* 手持ちのTUSD（2500TUSDくらい）分のBTCを買って売るだけ
* 4/27に大きく焼かれてからは、直近数時間の値動きが大きすぎるときはエントリーしないように対応
* 約定回数
    200～300回/日


これを今回の実装用に調整・具体化すると以下。

* 3秒毎にエントリーするかどうかを判定。

* エントリー条件
  * 前回のオーダーが残っていない（オーダー中および買いポジション(相当)がない）
  * 直前の値動きが大きくない
    * ３０分足を４本とり、平均の値幅が閾値以下。
* オーダーキャンセル条件
  * EXPIRE_TIME(600秒: 10分)以上約定しないオーダーはキャンセルする。
* 執行戦略
  * 買い
    * エントリー条件が揃ったら執行
    * 指値: 3秒足の終値 * (1-OFFSET)     // OFFSETは0.0005などの値
    * 数量: 0.001 BTC
  * 売り
    * 買いオーダーの約定が残っていたら執行
    * 指値：買いオーダー発行時の3秒足の終値 * (1+OFFSET)     // OFFSETは0.0005などの値
    * 数量：約定した買い注文と同数（=仮想敵なPosition）
    * その他：売りオーダーがExpireしてCancelされた場合、即、market_orderで投げうる。


In [ ]:
# rbot拡張ライブラリのインストール（２回目は不要です）
! pip install --upgrade pip
! pip install rbot

In [ ]:
# 関連ライブラリのインストール
# Polarsは0.20.0が必要です。
! pip install pyarrow
! pip install --upgrade polars
! pip install plotly
! pip install nbformat
! pip install numpy
! pip install pandas
! pip install json2html

# Agent（BOT）の実装

In [ ]:
class BasilAgent:
    def __init__(self):
        self.OFFSET = 0.00_05
        self.EXPIRE_TIME = 600    # 600[sec] = 10[min]
        self.ORDER_SIZE = 0.01
        self.RANGE = 300

    def on_init(self, session):
        session.clock_interval_sec = 3     # 3秒ごとに on_clock を呼び出す

    def on_clock(self, session, clock):
        if session.expire_order(self.EXPIRE_TIME):         # 期限切れの注文をキャンセルする.
            return                                          # 期限切れがあればリターン

        if session.buy_orders or session.sell_orders:   # 既に注文がある場合はリターン
            return

        # 1時間足のレンジを計算してログに出力する。レンジが大きい場合はトレードしない。
        ohlcv1h = session.ohlcv(60*60, 4)
        range = (ohlcv1h['high']-ohlcv1h['low']).mean()
        session.log_indicator("range", range)

        if self.RANGE < range:
            return

        ohlcv = session.ohlcv(3, 1)         # 3秒足を1本分取得。
        if len(ohlcv) < 1:                 # 3秒間に約定データがない場合リターン
            print("NO OHLCV DATA")
            return

        if session.position <= 0.001:    # ポジションが少ない場合は買い注文を出す。
            order_price = ohlcv['close'][-1] * (1 - self.OFFSET)
            print("BUY ORDER: ", order_price, self.ORDER_SIZE)
            session.limit_order('Buy', order_price, self.ORDER_SIZE - session.position)

        else:                       # ポジションがある場合は売り注文を出す。
            order_price = ohlcv['close'][-1] * (1 + self.OFFSET)
            print("SELL ORDER: ", order_price, session.position)
            session.limit_order('Sell', order_price, session.position)

    def on_update(self, session, updated_order):
        # 売りオーダーが期限切れされた場合には、成り行きで売り注文を出す（ロスカット）
        if updated_order.status == 'Canceled':
            if updated_order.side == 'Sell':
                session.market_order('Sell', updated_order.remaining_size)



## バックテスト

In [ ]:
from rbot import BinanceConfig, BinanceMarket
from rbot import Runner
from rbot import NOW, DAYS

rbot version:  0.2.13
!!! ABSOLUTELY NO WARRANTY !!!
!!!  USE AT YOUR OWN RISK  !!!
See document at https://github.com/yasstake/rusty-bot
 All rights reserved. (c) 2022-2023 rbot developers / yasstake


In [ ]:
config = BinanceConfig.BTCUSDT
market = BinanceMarket(config)

# バックテストの期間を設定
BACKTEST_DAYS = 2

# バックテスト用に過去データをダウンロード
market.download(
    ndays=BACKTEST_DAYS,          # 過去１日分のデータをダウンロード
    verbose=True,
    archive_only=True
)

0

In [ ]:
agent = BasilAgent()        # 実行対象のエージェントを指定
runner = Runner()           # 実行モジュール Runner を作成

In [ ]:
session = runner.back_test(
    agent=agent,                    # エージェントを指定
    market=market,                  # マーケットを指定
    start_time=NOW()-DAYS(BACKTEST_DAYS), # 開始時刻を指定
    end_time=0,                     # 終了時刻を指定(0を指定すると最後まで実行)
    # execute_time=60*60*24,        # 実行時間を指定(指定しない、もしくは0を指定すると最後まで実行)
    verbose=True,                   # 実行の進捗を表示
    # log_file="./bot.log"            # ログファイルを指定(指定しないとファイルは作られない）
)


BUY ORDER:  42065.446755 0.01
SELL ORDER:  42082.900935 0.01
BUY ORDER:  42065.046955 0.01
SELL ORDER:  42081.03 0.01
BUY ORDER:  42060.05945 0.01
SELL ORDER:  42060.529755 0.01
SELL ORDER:  41917.48827 0.01
SELL ORDER:  42085.59228 0.01
BUY ORDER:  42064.957 0.01
BUY ORDER:  42108.925005 0.01
SELL ORDER:  42122.71083 0.01
SELL ORDER:  42064.8219 0.01
BUY ORDER:  42046.94601000001 0.01
BUY ORDER:  42101.63865 0.01
BUY ORDER:  42152.263325 0.01
SELL ORDER:  42165.322125 0.01
BUY ORDER:  42202.398245000004 0.01
SELL ORDER:  42222.200549999994 0.01
SELL ORDER:  42144.851895 0.01
BUY ORDER:  42124.737095 0.01
BUY ORDER:  42224.867005 0.01
SELL ORDER:  42242.580734999996 0.01
BUY ORDER:  42228.865005 0.01
SELL ORDER:  42247.21305 0.01
SELL ORDER:  42233.716305 0.01
SELL ORDER:  42191.074994999995 0.01
BUY ORDER:  42170.894005 0.01
BUY ORDER:  42169.974465 0.01
SELL ORDER:  42182.66079 0.01
BUY ORDER:  42162.04843 0.01
SELL ORDER:  42157.898414999996 0.01
BUY ORDER:  42146.39626000001 0.01
S

In [ ]:
log = session.log
orders = log.orders
orders.tail(5)

log_id,symbol,update_time,create_time,status,order_id,client_order_id,order_side,order_type,order_price,order_size,remain_size,transaction_id,execute_price,execute_size,quote_vol,commission,commission_asset,is_maker,message,commission_home,commission_foreign,home_change,foreign_change,free_home_change,free_foreign_change,lock_home_change,lock_foreign_change,open_position,close_position,position,profit,fee,total_profit,sum_profit
i64,str,datetime[μs],datetime[μs],str,str,str,str,str,f64,f64,f64,str,f64,f64,f64,f64,str,bool,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
997,"""BTCUSDT""",2023-12-16 23:45:23.154,2023-12-16 23:41:42,"""PartiallyFille…","""20231217T1515-…","""20231217T1515-…","""Buy""","""Limit""",42210.49,0.01,0.00013,"""20231217T1515-…",42210.49,0.00944,398.467026,0.0,"""""",true,"""""",0.0,0.0,-398.467026,0.00944,0.0,0.00944,-398.467026,0.0,0.00944,0.0,0.00987,0.0,0.039847,-0.039847,-12.257656
998,"""BTCUSDT""",2023-12-16 23:45:23.777,2023-12-16 23:41:42,"""Filled""","""20231217T1515-…","""20231217T1515-…","""Buy""","""Limit""",42210.49,0.01,0.0,"""20231217T1515-…",42210.49,0.00013,5.4873637,0.0,"""""",true,"""""",0.0,0.0,-5.487364,0.00013,0.0,0.00013,-5.487364,0.0,0.00013,0.0,0.01,0.0,0.000549,-0.000549,-12.258205
999,"""BTCUSDT""",2023-12-16 23:45:24.421,2023-12-16 23:45:24,"""New""","""20231217T1515-…","""20231217T1515-…","""Sell""","""Limit""",42229.1,0.01,0.01,"""""",0.0,0.0,0.0,0.0,"""""",true,"""""",0.0,0.0,0.0,0.0,0.0,-0.01,0.0,0.01,0.0,0.0,0.01,0.0,0.0,0.0,-12.258205
1000,"""BTCUSDT""",2023-12-16 23:50:31.409,2023-12-16 23:45:24,"""Filled""","""20231217T1515-…","""20231217T1515-…","""Sell""","""Limit""",42229.1,0.01,0.0,"""20231217T1515-…",42229.1,0.01,422.291,0.0,"""""",true,"""""",0.0,0.0,422.291,-0.01,422.291,-0.01,0.0,-0.01,0.0,0.01,0.0,0.1861,0.0422291,0.1438709,-12.114334
1001,"""BTCUSDT""",2023-12-16 23:50:33.079,2023-12-16 23:50:33,"""New""","""20231217T1515-…","""20231217T1515-…","""Buy""","""Limit""",42208.07,0.01,0.01,"""""",0.0,0.0,0.0,0.0,"""""",true,"""""",0.0,0.0,0.0,0.0,-422.0807,0.0,422.0807,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-12.114334


In [ ]:
# オーダー状況分析
# あまりにCancelが多い場合は、Expireの時間やオーダーのOFFSETを調整する必要がある。
orders.group_by(['order_side', 'status']).count()

order_side,status,count
str,str,u32
"""Buy""","""New""",146
"""Sell""","""Canceled""",54
"""Buy""","""Filled""",84
"""Buy""","""PartiallyFille…",235
"""Sell""","""PartiallyFille…",199
"""Buy""","""Canceled""",61
"""Sell""","""Filled""",84
"""Sell""","""New""",138


In [ ]:
# 最終想定損益
orders['sum_profit'][-1]

-12.11433362000001

In [ ]:
# 手数料なし想定損益
orders['profit'].sum()

-5.019

In [ ]:

# ローソク足データの作成
ohlcv = market.ohlcv(
    runner.start_timestamp,     # バックテストの開始時刻
    runner.last_timestamp,    # バックテストの終了時刻
    30                          # 30秒足
)
ohlcv.tail(5)

timestamp,open,high,low,close,volume,count
datetime[μs],f64,f64,f64,f64,f64,u32
2023-12-16 23:57:30,42279.49,42279.49,42276.0,42276.01,2.00294,142
2023-12-16 23:58:00,42276.01,42277.24,42271.92,42277.24,4.7609,300
2023-12-16 23:58:30,42277.23,42278.03,42277.23,42278.02,0.93507,141
2023-12-16 23:59:00,42278.02,42278.03,42278.02,42278.02,1.70851,150
2023-12-16 23:59:30,42278.02,42278.03,42278.02,42278.02,2.04203,138


In [ ]:
# session.log_indicator('range', value)で保存したデータは以下で取得できます。戻り値はpolars.DataFrameです。
log['range'].tail(5)

timestamp,range
datetime[μs],f64
2023-12-16 23:27:21,154.312
2023-12-16 23:31:36,154.312
2023-12-16 23:41:42,144.64
2023-12-16 23:45:24,142.308
2023-12-16 23:50:33,140.64


In [ ]:
# バックテスト結果の可視化

# 数千件のデータを可視化するときは、plotlyを使うと便利。
# https://plotly.com/python/
# ただし約定データが万を超えると、ブラウザが重くなるので注意。一部データ切り取りなどが必要。

import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.1, row_heights=[0.1, 0.1, 0.1, 0.6],
                    subplot_titles=("range", "psudo-position", "psudo-profit", "candlestick"))

# row 1 (indicator)
spread = log['range']
fig.add_trace(go.Scatter(x=spread['timestamp'], y=spread['range'], name="range"), row=1, col=1)

# row 2 (position)
fig.add_trace(go.Scatter(x=orders['update_time'], y=orders['position'], name="position", line=dict(shape='hv')), row=2, col=1)

# row 3 (profit)
profit = orders['sum_profit']
fig.add_trace(go.Scatter(x=orders['update_time'], y=orders['sum_profit'], name="profit", line=dict(shape='hv')), row=3, col=1)

# row 4 (candlestick)
fig.add_candlestick(x=ohlcv['timestamp'], open=ohlcv['open'], high=ohlcv['high'], low=ohlcv['low'], close=ohlcv['close'], row=4, col=1)

# row 4 (order)
buy_orders = orders.filter((orders['order_side'] == 'Buy') & (orders['status'] == 'New'))
fig.add_trace(go.Scatter(x=buy_orders['update_time'], y=buy_orders['order_price'], mode='markers', marker=dict(symbol='arrow-up', color='red', size=10), name="buy"), row=4, col=1)

buy_orders = orders.filter((orders['order_side'] == 'Buy') & (orders['status'] == 'Filled'))
fig.add_trace(go.Scatter(x=buy_orders['update_time'], y=buy_orders['order_price'], mode='markers', marker=dict(symbol='cross-thin-open', color='red', size=10), name="buy filled"), row=4, col=1)

buy_orders = orders.filter((orders['order_side'] == 'Buy') & (orders['status'] == 'Canceled'))
fig.add_trace(go.Scatter(x=buy_orders['update_time'], y=buy_orders['order_price'], mode='markers', marker=dict(symbol='x-thin-open', color='red', size=10), name="buy canceled"), row=4, col=1)


sell_orders = orders.filter((orders['order_side'] == 'Sell') & (orders['status'] == 'New'))
fig.add_trace(go.Scatter(x=sell_orders['update_time'], y=sell_orders['order_price'], mode='markers', marker=dict(symbol='arrow-down', color='blue', size=10), name="sell"), row=4, col=1)

sell_orders = orders.filter((orders['order_side'] == 'Sell') & (orders['status'] == 'Filled'))
fig.add_trace(go.Scatter(x=sell_orders['update_time'], y=sell_orders['order_price'], mode='markers', marker=dict(symbol='cross-thin-open', color='blue', size=10), name="sell filled"), row=4, col=1)

sell_orders = orders.filter((orders['order_side'] == 'Sell') & (orders['status'] == 'Canceled'))
fig.add_trace(go.Scatter(x=sell_orders['update_time'], y=sell_orders['order_price'], mode='markers', marker=dict(symbol='x-thin-open', color='blue', size=10), name="sell canceled"), row=4, col=1)

fig.update_layout(height=800, title_text="Backtest Result")


# フォーワードテスト

ここからは、WebSocketでリアルタイムデータを使います。そのためColabでは動きません。ローカルにjupyter環境を準備して実行してください。

In [ ]:
runner = Runner()

session = runner.dry_run(
    agent=agent,
    market=market,
    log_memory=True,
    execute_time=60*5,     # 60x5=5分間
    verbose=True,
    # log_file="./bot.log"            # ログファイルを指定(指定しないとファイルは作られない）
)

In [ ]:
log = session.log

In [ ]:
orders = log.orders
orders.tail(5)

In [ ]:
ohlcv = market.ohlcv(runner.start_timestamp, runner.last_timestamp, 10)
ohlcv

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import plot

fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.1, row_heights=[0.1, 0.1, 0.1, 0.6],
                    subplot_titles=("range", "psudo-position", "psudo-profit", "candlestick"))

# row 1 (indicator)
spread = log['range']
fig.add_trace(go.Scatter(x=spread['timestamp'], y=spread['range'], name="range"), row=1, col=1)

# row 2 (position)
fig.add_trace(go.Scatter(x=orders['update_time'], y=orders['position'], name="position", line=dict(shape='hv')), row=2, col=1)

# row 3 (profit)
profit = orders['sum_profit']
fig.add_trace(go.Scatter(x=orders['update_time'], y=orders['sum_profit'], name="profit", line=dict(shape='hv')), row=3, col=1)

# row 4 (candlestick)
fig.add_candlestick(x=ohlcv['timestamp'], open=ohlcv['open'], high=ohlcv['high'], low=ohlcv['low'], close=ohlcv['close'], row=4, col=1)

# row 4 (order)
buy_orders = orders.filter((orders['order_side'] == 'Buy') & (orders['status'] == 'New'))
fig.add_trace(go.Scatter(x=buy_orders['update_time'], y=buy_orders['order_price'], mode='markers', marker=dict(symbol='arrow-up', color='red', size=10), name="buy"), row=4, col=1)

buy_orders = orders.filter((orders['order_side'] == 'Buy') & (orders['status'] == 'Filled'))
fig.add_trace(go.Scatter(x=buy_orders['update_time'], y=buy_orders['order_price'], mode='markers', marker=dict(symbol='cross-thin-open', color='red', size=10), name="buy filled"), row=4, col=1)

buy_orders = orders.filter((orders['order_side'] == 'Buy') & (orders['status'] == 'Canceled'))
fig.add_trace(go.Scatter(x=buy_orders['update_time'], y=buy_orders['order_price'], mode='markers', marker=dict(symbol='x-thin-open', color='red', size=10), name="buy canceled"), row=4, col=1)


sell_orders = orders.filter((orders['order_side'] == 'Sell') & (orders['status'] == 'New'))
fig.add_trace(go.Scatter(x=sell_orders['update_time'], y=sell_orders['order_price'], mode='markers', marker=dict(symbol='arrow-down', color='blue', size=10), name="sell"), row=4, col=1)

sell_orders = orders.filter((orders['order_side'] == 'Sell') & (orders['status'] == 'Filled'))
fig.add_trace(go.Scatter(x=sell_orders['update_time'], y=sell_orders['order_price'], mode='markers', marker=dict(symbol='cross-thin-open', color='blue', size=10), name="sell filled"), row=4, col=1)

sell_orders = orders.filter((orders['order_side'] == 'Sell') & (orders['status'] == 'Canceled'))
fig.add_trace(go.Scatter(x=sell_orders['update_time'], y=sell_orders['order_price'], mode='markers', marker=dict(symbol='x-thin-open', color='blue', size=10), name="sell canceled"), row=4, col=1)

fig.update_layout(height=800, title_text="Forwadtest Result")
